In [ ]:
import pandas

df = pandas.read_csv('./data/model_lantus_3.5.csv')
df.tail(10)

In [ ]:
from pymc3 import *
import theano.tensor as T
import matplotlib.pyplot as plt
%matplotlib inline

glucose_diff = df['glucose_diff']
basal_given = df['lantus']
mealcarbs = df['mealcarbs_last_meal']
humalog = df['humalog_last_meal']
timediff_last_mealcarbs = df['time_diff_minutes']
timediff_last_glucose = df['time_diff_minutes']
timediff_last_humalog_last_glucose = df['time_diff_minutes']

import theano.tensor as T

In [ ]:
# Assume that basal insulin is accurate dosage (this is a large assumption),
# We can try and incorporate this later with another variable.
with Model() as model: # model specifications in PyMC3 are wrapped in a with-statement
    # Define priors
    # Define priors
    # sigma = HalfCauchy('sigma', beta=5, testval=1.)
    
    time_of_absorb_carbs = 120
    #time_of_absorb_carbs = Uniform('time_of_absorb_carbs', lower=30, upper=120)
    
    #time_of_absorb_humalog = Uniform('time_of_absorb_humalog', lower=180, upper=240)
    time_of_absorb_humalog = 180

    CR = Uniform('CR',lower=1, upper=200)
    CF = Uniform('CF', lower=1, upper=500)

    percent_absorb_carbs = T.minimum(timediff_last_mealcarbs, time_of_absorb_carbs) / time_of_absorb_carbs
    percent_absorb_humalog = T.minimum(timediff_last_humalog_last_glucose, time_of_absorb_humalog) / time_of_absorb_humalog
 
    #basal_actual = Normal('basal_actual', mu=6, sd=2)

    # Define likelihood #+ CF *.8* (basal_actual - basal_given) * 1/(24*60) * timediff_last_glucose
    likelihood = Normal('glucose_diff', mu=CF/CR * mealcarbs * percent_absorb_carbs \
                        - CF * humalog * percent_absorb_humalog \
                        ,sd=50, observed=glucose_diff)
    
    # Inference!
    trace = sample(6000, tune=2000, progressbar=True, start={'CF': 150, 'CR': 50}) # draw posterior samples using NUTS sampling
    

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(7, 7))

traceplot(trace[100:])
plt.tight_layout();